In [ ]:
import pandas as pd
import patsy
import numpy as np
from statsmodels.formula.api import ols
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [ ]:
df = pd.read_csv("dentalplan.txt",sep=r"\s+",header=None,names=["Pain_Relief", "Block", "Drug", "Acupuncture"])

df

In [ ]:
D_block = np.array([
    [1, 0],  
    [1, 0],  
    [0, 1],  
    [0, 1]   
])
X1 = np.kron(np.identity(8), D_block)
X1

In [ ]:
A_block = np.array([
    [1, 0], 
    [0, 1],  
    [1, 0],  
    [0, 1] 
])

X1 = np.kron(np.identity(8), D_block)
X1
X2 = np.kron(np.identity(8), A_block)
X2

In [ ]:
X = np.hstack([X1, X2])
X

In [ ]:
y, X = patsy.dmatrices('Pain_Relief ~ C(Drug) * C(Acupuncture) + C(Block)', data=df)
beta_hat = np.linalg.inv(X.T @ X) @ X.T @ y
beta_hat

In [ ]:
model_blocked = ols('Pain_Relief ~ C(Drug) * C(Acupuncture) + C(Block)', data=df).fit()
print(model_blocked.summary())

In [ ]:
print(model_blocked.params)

In [ ]:

residuals = model_blocked.resid
fitted = model_blocked.fittedvalues


df_resid = pd.DataFrame({
    "Fitted": fitted,
    "Residuals": residuals
})

print(df_resid.head())

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(x=fitted, y=residuals, s=70, color="steelblue", edgecolor="black")
plt.axhline(0, color="red", linestyle="--", linewidth=1)
plt.xlabel(r"Fitted values ($\hat{y}$)", fontsize=12)
plt.ylabel(r"Residuals ($\epsilon_i$)", fontsize=12)
plt.title("Residuals vs Fitted Values — Randomized Block Model", fontsize=13)
plt.show();

In [ ]:
anova_table = sm.stats.anova_lm(model_blocked, typ=2)
print(anova_table)

In [ ]:
sns.pointplot(
    data=df, x="Drug", y="Pain_Relief",
    hue="Acupuncture", errorbar=('ci', 95), dodge=True, capsize=0.1,
    markers=["o", "s"]
)
plt.title("Interaction Plot: Drug X Acupuncture (95% CI)")
plt.ylabel("Mean Pain Relief ± 95% CI")
plt.show();


In [ ]:
means_drug = df.groupby("Drug")["Pain_Relief"].mean()
print(means_drug)


means_acup = df.groupby("Acupuncture")["Pain_Relief"].mean()
print(means_acup)

In [ ]:
# Drug effect (alpha_1 - alpha_2)
diff_drug = means_drug[2] - means_drug[1]  

# Acupuncture effect (beta_1 - beta_2)
diff_acup = means_acup[2] - means_acup[1] 

print(f"Estimated Drug effect (alpha1 - alpha2): {diff_drug:.3f}")
print(f"Estimated Acupuncture effect (beta1 - beta2): {diff_acup:.3f}")
